# Part_C : KMeans Clustering

In [1]:
import pandas as pd
import numpy as np
import os
df=pd.read_csv('processed_dataset.csv')
df.head()

foolishness  hath  wholesome  takest  feelings  anger  vaivaswata  matrix  \
0          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
1          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
2          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
3          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   
4          0.0   0.0        0.0     0.0       0.0    0.0         0.0     0.0   

   kindled  convict  ...  thinkest  modern  reigned  sparingly  visual  \
0      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
1      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
2      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
3      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   
4      0.0      0.0  ...       0.0     0.0      0.0        0.0     0.0   

   thoughts  illumines  attire  explains  output_label  
0       0.0        0.0     0.0       0.0      Buddhism  
1       0.0        0.0     0.0       0.0      Buddhism  
2       0.0        0.0     0.0       0.0      Buddhism  
3       0.0        0.0     0.0       0.0      Buddhism  
4       0.0        0.0     0.0       0.0      Buddhism  

[5 rows x 8267 columns]

In [2]:
# for array of values
df_array=df.drop(['output_label'],axis=1).values
# for prestored cosine similarity values
dot_array=np.dot(df_array,df_array.T)

In [3]:
num_clusters_required=8 # REQUIRED NO. OF CLUSTERS DECLARED AT THE BEGINNING

In [4]:
def vector_distance(v1,v2): # v1,v2: unit l2-normalised vectors, FUNCTION RETURNS DISTANCE BETWEEN THE VECTORS
    cosine_sim=np.dot(v1,v2)
    return np.exp(-cosine_sim)

In [5]:
# RETURN CLUSTERS OF DOCUMENTS GIVEN CENTROIDS OF CLUSTERS
def assign_clusters(tfidf_mat,centroids,cluster_assignment,num_docs):
    f=0 # f=0 denotes no change from previous cluster assignment, so algorithm terminates, else it continues (f=1)
    cluster_docs={}
    for i in range(num_docs):
        doc_vector=tfidf_mat[i]
        min_dist=-1
        for j in range(num_clusters_required):
            centroid_vector=centroids[j]
            dist=vector_distance(doc_vector,centroid_vector) # DISTANCE OF DOCUMENT I FROM CLUSTER J
            if min_dist<0:
                min_dist=dist
                closest_centroid=j #UPDATE CLOSEST CLUSTER CENTROID
                continue
            else:
                if min_dist>dist:
                    min_dist=dist
                    closest_centroid=j #UPDATE CLOSEST CLUSTER CENTROID
        if closest_centroid not in cluster_docs.keys(): #IF THE CLUSTER INDEX EXISTS IN KEY LIST
            cluster_docs[closest_centroid]=[i]
        else:
            cluster_docs[closest_centroid].append(i)
        if cluster_assignment[i]!=closest_centroid: # CHANGE FROM PREVIOUS ASSIGNMENT OF DOC TO CLUSTER, ASSIGNED TO SOME OTHER CLUSTER, SO f=1
            cluster_assignment[i]=closest_centroid
            f=1
    return f,cluster_docs

In [6]:
def update_centroids(cluster_docs,centroids,tfidf_mat):
    for key in sorted(cluster_docs):
        centroids[key]=np.mean(tfidf_mat[cluster_docs[key]],axis=0) # CENTROID IS THE MEAN ACROSS ALL FEATURES(ACROSS COLUMNS)
    return centroids

In [7]:
# KMEANS CLUSTERING ALGORITHM
def kmeans(tfidf_mat,num_clusters_required):
    num_docs=tfidf_mat.shape[0]
    num_features=tfidf_mat.shape[1]
    centroids=np.random.rand(num_clusters_required,num_features) # CENTROIDS RANDOMLY INITIALISED
    cluster_assignment=[] #FIRST, CLUSTER ASSIGNMENT HAS TO BE DONE
    cluster_docs={} # CONTAINS DOCUMENT INDICES OF EVERY CLUSTER
    for i in range(num_docs):
        doc_vector=tfidf_mat[i]
        min_dist=-1
        for j in range(num_clusters_required):
            centroid_vector=centroids[j]
            dist=vector_distance(doc_vector,centroid_vector)
            if min_dist<0:
                min_dist=dist
                closest_centroid=j
                continue
            else:
                if min_dist>dist:
                    min_dist=dist
                    closest_centroid=j
        cluster_assignment.append(closest_centroid)
        if closest_centroid not in cluster_docs.keys():
            cluster_docs[closest_centroid]=[i]
        else:
            cluster_docs[closest_centroid].append(i)
    f=0
    iterations=0
    while iterations<10000: # REPEAT UNTIL CONVERGENCE OF CLUSTERS
        centroids=update_centroids(cluster_docs,centroids,tfidf_mat) # M-STEP (CENTROID UPDATE)
        f,cluster_docs=assign_clusters(tfidf_mat,centroids,cluster_assignment,num_docs) # E-STEP (CLUSTER ASSIGNMENT)
        if f==0: # No change in cluster assignments, algorithm terminates
            break
        iterations+=1
    ordered_cluster_docs=sorted(cluster_docs.items(),key = lambda x : x[1]) #SORT CLUSTERS BY INDICES OF THEIR COMPONENT DOCUMENTS
    return ordered_cluster_docs

In [8]:
def write(ordered_cluster_docs,filename): # filename, including extension .txt, WRITE THE CLUSTER INFORMATION TO FILE
    c=0
    with open('%s'%(filename),'w') as f:
        for item in ordered_cluster_docs:
            f.write(','.join([str(idx) for idx in item[1]]))
            c+=1
            if c<num_clusters_required:
                f.write('\n')

In [13]:
tfidf_mat=df_array
ordered_cluster_docs=kmeans(tfidf_mat,num_clusters_required) # GET ORDERED CLUSTERS FROM KMEANS
write(ordered_cluster_docs,'kmeans.txt') # WRITE CLUSTERS TO FILE

In [10]:
len(ordered_cluster_docs)

8

In [11]:
for c in range(8):
    print(ordered_cluster_docs[c])

(5, [0, 25, 47, 54, 62, 73, 82, 83, 108, 114, 123, 125, 139, 143, 145, 210, 217, 219, 224, 225, 252, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 286, 307, 369, 412, 413])
(0, [1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 23, 24, 26, 27, 28, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 58, 60, 113, 132, 188, 194, 201, 202, 213, 228, 229, 232, 239, 242, 244, 248, 254, 266, 285, 301, 305, 306, 317, 320, 330, 354, 358, 365, 382, 383, 385, 386, 391, 400, 408, 410, 416, 431, 432, 433, 442, 446, 462, 472])
(7, [4, 29, 30, 33, 63, 70, 71, 85, 86, 87, 90, 98, 105, 110, 115, 117, 118, 129, 131, 190, 192, 193, 197, 198, 199, 200, 203, 206, 220, 221, 222, 227, 233, 234, 236, 237, 255, 311, 384, 399, 402, 448, 449])
(4, [6, 17, 22, 52, 57, 65, 107, 121, 180, 184, 215, 223, 231, 287, 296, 313, 315, 321, 327, 329, 331, 332, 333, 336, 340, 355, 360, 381, 388, 392, 394, 398, 403, 405, 407, 409, 414, 423, 440, 447, 454, 455, 460, 461, 466, 474])
(6, [10,